In [11]:
import os
import numpy as np
import cv2

# ------------------------------
# ENTER YOUR PARAMETERS HERE:
ARUCO_DICT = cv2.aruco.DICT_6X6_250
SQUARES_VERTICALLY = 7
SQUARES_HORIZONTALLY = 5
SQUARE_LENGTH = 0.02
MARKER_LENGTH = 0.01
LENGTH_PX = 640   # total length of the page in pixels
MARGIN_PX = 20    # size of the margin in pixels
SAVE_NAME = 'ChArUco_Marker.png'
# ------------------------------

def create_and_save_new_board():
    dictionary = cv2.aruco.getPredefinedDictionary(ARUCO_DICT)
    board = cv2.aruco.CharucoBoard((SQUARES_VERTICALLY, SQUARES_HORIZONTALLY), SQUARE_LENGTH, MARKER_LENGTH, dictionary)
    size_ratio = SQUARES_HORIZONTALLY / SQUARES_VERTICALLY
    img = cv2.aruco.CharucoBoard.generateImage(board, (LENGTH_PX, int(LENGTH_PX*size_ratio)), marginSize=MARGIN_PX)
    #cv2.imshow("img", img)
    #cv2.waitKey(2000)
    cv2.imwrite(SAVE_NAME, img)

create_and_save_new_board()

In [12]:
import cv2
import os

# Create the 'images' directory if it doesn't exist
if not os.path.exists('images'):
    os.makedirs('images')

cap = cv2.VideoCapture(0)
cap.set(3, 640)  # Set width
cap.set(4, 480)  # Set height

counter = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    cv2.imshow('frame', frame)

    key = cv2.waitKey(2) & 0xFF
    if key == 27:  # Esc key to save the image
        cv2.imwrite(f'images/{counter}.jpg', frame)
        print(f"Image saved as images/{counter}.jpg")
        counter += 1
    elif key == ord('q'):  # 'q' key to quit
        print("Quitting...")
        break

cap.release()
cv2.destroyAllWindows()


Image saved as images/0.jpg
Quitting...


In [2]:
ARUCO_DICT = cv2.aruco.DICT_6X6_250
SQUARES_VERTICALLY = 7
SQUARES_HORIZONTALLY = 5
SQUARE_LENGTH = 0.02
MARKER_LENGTH = 0.01
# ...
PATH_TO_YOUR_IMAGES = '/images'
# ------------------------------
def get_calibration_parameters(img_dir):
    print(img_dir)
    # Define the aruco dictionary, charuco board and detector
    dictionary = cv2.aruco.getPredefinedDictionary(ARUCO_DICT)
    board = cv2.aruco.CharucoBoard((SQUARES_VERTICALLY, SQUARES_HORIZONTALLY), SQUARE_LENGTH, MARKER_LENGTH, dictionary)
    params = cv2.aruco.DetectorParameters()
    detector = cv2.aruco.ArucoDetector(dictionary, params)
    
    # Load images from directory
    image_files = [os.path.join(img_dir, f) for f in os.listdir(img_dir) if f.endswith(".jpg")]
    print(image_files)
    all_charuco_ids = []
    all_charuco_corners = []

    # Loop over images and extraction of corners
    for image_file in image_files:
        print(image_file)
        image = cv2.imread(image_file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        imgSize = image.shape
        image_copy = image.copy()
        marker_corners, marker_ids, rejectedCandidates = detector.detectMarkers(image)
        
        if len(marker_ids) > 0: # If at least one marker is detected
            # cv2.aruco.drawDetectedMarkers(image_copy, marker_corners, marker_ids)
            ret, charucoCorners, charucoIds = cv2.aruco.interpolateCornersCharuco(marker_corners, marker_ids, image, board)

            if charucoIds is not None and len(charucoCorners) > 3:
                all_charuco_corners.append(charucoCorners)
                all_charuco_ids.append(charucoIds)
    
    # Calibrate camera with extracted information
    result, mtx, dist, rvecs, tvecs = cv2.aruco.calibrateCameraCharuco(all_charuco_corners, all_charuco_ids, board, imgSize, None, None)
    return mtx, dist

In [3]:
import json
SENSOR = 'monochrome'
LENS = 'kowa_f12mm_F1.8'
OUTPUT_JSON = 'calibration.json'

mtx, dist = get_calibration_parameters(img_dir='./images/')
data = {"sensor": SENSOR, "lens": LENS, "mtx": mtx.tolist(), "dist": dist.tolist()}

with open(OUTPUT_JSON, 'w') as json_file:
    json.dump(data, json_file, indent=4)

print(f'Data has been saved to {OUTPUT_JSON}')

./images/
['./images/0.jpg', './images/1.jpg', './images/2.jpg', './images/3.jpg', './images/4.jpg', './images/5.jpg', './images/6.jpg', './images/7.jpg']
./images/0.jpg
./images/1.jpg
./images/2.jpg
./images/3.jpg
./images/4.jpg
./images/5.jpg
./images/6.jpg
./images/7.jpg
Data has been saved to calibration.json


In [13]:
json_file_path = './calibration.json'

with open(json_file_path, 'r') as file: # Read the JSON file
    json_data = json.load(file)

mtx = np.array(json_data['mtx'])
dst = np.array(json_data['dist'])

In [15]:
image_path = '0.jpg'
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
h,  w = image.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dst, (w,h), 1, (w,h))
image = cv2.undistort(image, mtx, dst, None, newcameramtx)

In [16]:
all_charuco_ids = []
all_charuco_corners = []

dictionary = cv2.aruco.getPredefinedDictionary(ARUCO_DICT)
board = cv2.aruco.CharucoBoard((SQUARES_VERTICALLY, SQUARES_HORIZONTALLY), SQUARE_LENGTH, MARKER_LENGTH, dictionary)
params = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(dictionary, params)

marker_corners, marker_ids, rejectedCandidates = detector.detectMarkers(image)
if marker_ids is not None and len(marker_ids) > 0: # If at least one marker is detected
    # cv2.aruco.drawDetectedMarkers(image_copy, marker_corners, marker_ids)
    ret, charucoCorners, charucoIds = cv2.aruco.interpolateCornersCharuco(marker_corners, marker_ids, image, board)
    if charucoCorners is not None and charucoIds is not None and len(charucoCorners) > 3:
        all_charuco_corners.append(charucoCorners)
        all_charuco_ids.append(charucoIds)

    retval, rvec, tvec = cv2.aruco.estimatePoseCharucoBoard(np.array(all_charuco_corners)[0], np.array(all_charuco_ids)[0], board, np.array(mtx), np.array(dst), np.empty(1), np.empty(1))

    Zx, Zy, Zz = tvec[0][0], tvec[1][0], tvec[2][0]
    fx, fy = mtx[0][0], mtx[1][1]

    print(f'Zz = {Zz}\nZy = {Zy}\nZx = {Zx}\nfx = {fx}\nfy = {fy}')

IndexError: index 0 is out of bounds for axis 0 with size 0

In [9]:
def perspective_function(x, Z, f): 
    return x*Z/f

nb_pixels = 192
print(perspective_function(nb_pixels, Zz, fx))

0.15052876578470026


In [17]:
import cv2
import numpy as np
import json

# ------------------------------
# Parameters
ARUCO_DICT = cv2.aruco.DICT_5X5_100
MARKER_LENGTH_METERS = 0.04  # Actual side length of the marker in meters (adjust as per your marker size)
CALIBRATION_JSON = 'calibration.json'  # JSON file containing the camera calibration
# ------------------------------

# Load camera calibration data
def load_calibration(calibration_file):
    with open(calibration_file, 'r') as file:
        data = json.load(file)
    mtx = np.array(data['mtx'])
    dist = np.array(data['dist'])
    return mtx, dist

# Function to detect and measure the ArUco marker
def measure_aruco_marker(image_path, calibration_file):
    # Load the camera calibration data
    mtx, dist = load_calibration(calibration_file)
    
    # Load the input image
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Set up the ArUco dictionary and parameters
    dictionary = cv2.aruco.getPredefinedDictionary(ARUCO_DICT)
    params = cv2.aruco.DetectorParameters()
    detector = cv2.aruco.ArucoDetector(dictionary, params)
    
    # Detect markers
    marker_corners, marker_ids, rejected_candidates = detector.detectMarkers(gray)
    
    if marker_ids is not None and len(marker_corners) > 0:
        for corners, marker_id in zip(marker_corners, marker_ids):
            # Get the corner points of the detected marker
            corners = corners[0]  # Extract the array of corners
            # Draw the marker outline on the image
            cv2.polylines(image, [corners.astype(int)], True, (0, 255, 0), 2)
            
            # Calculate the distance between the first two corners
            # Assuming square markers, calculate physical size using marker length
            pixel_length = np.linalg.norm(corners[0] - corners[1])
            focal_length = mtx[0, 0]  # Approximation using fx (focal length in pixels)
            distance = (MARKER_LENGTH_METERS * focal_length) / pixel_length
            
            print(f"Marker ID: {marker_id[0]}")
            print(f"Measured Distance (m): {distance:.4f}")
            print(f"Pixel Length: {pixel_length:.2f}")
            
            # Annotate the image
            center = corners.mean(axis=0).astype(int)
            cv2.putText(image, f"ID: {marker_id[0]}", tuple(center), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
            cv2.putText(image, f"Dist: {distance:.2f}m", (center[0], center[1] + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
    else:
        print("No markers detected.")
    
    # Display and save the annotated image
    cv2.imshow("Measured Marker", image)
    cv2.imwrite("measured_marker.png", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Measure a marker in an image
image_path = '0.jpg'  # Replace with the path to your ArUco marker image
measure_aruco_marker(image_path, CALIBRATION_JSON)


Marker ID: 2
Measured Distance (m): 0.3634
Pixel Length: 72.01


In [1]:
import cv2
import numpy as np
import json

# Load calibration data
json_file_path = './calibration.json'
with open(json_file_path, 'r') as file: 
    json_data = json.load(file)

mtx = np.array(json_data['mtx'])
dist = np.array(json_data['dist'])

# Charuco board parameters
ARUCO_DICT = cv2.aruco.DICT_6X6_250
SQUARES_VERTICALLY = 7
SQUARES_HORIZONTALLY = 5
SQUARE_LENGTH = 0.02
MARKER_LENGTH = 0.01

dictionary = cv2.aruco.getPredefinedDictionary(ARUCO_DICT)
board = cv2.aruco.CharucoBoard((SQUARES_VERTICALLY, SQUARES_HORIZONTALLY), SQUARE_LENGTH, MARKER_LENGTH, dictionary)
params = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(dictionary, params)

# Global variables to store the fixed position
# Load fixed position from JSON if available
try:
    with open("fixed_position.json", "r") as json_file:
        fixed_data = json.load(json_file)
        fixed_rvec = np.array(fixed_data["fixed_rvec"])
        fixed_tvec = np.array(fixed_data["fixed_tvec"])
    print("Fixed position loaded from file.")
except FileNotFoundError:
    fixed_rvec = None
    fixed_tvec = None
    print("No fixed position found. Please save a fixed position.")


def save_fixed_position(frame):
    global fixed_rvec, fixed_tvec

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    marker_corners, marker_ids, _ = detector.detectMarkers(gray)

    if len(marker_ids) > 0:
        _, charuco_corners, charuco_ids = cv2.aruco.interpolateCornersCharuco(marker_corners, marker_ids, gray, board)
        if charuco_ids is not None and len(charuco_corners) > 3:
            # Initialize rvec and tvec
            rvec = np.zeros((3, 1), dtype=np.float64)
            tvec = np.zeros((3, 1), dtype=np.float64)

            retval, rvec, tvec = cv2.aruco.estimatePoseCharucoBoard(
                charuco_corners, charuco_ids, board, mtx, dist, rvec, tvec
            )

            if retval:
                fixed_rvec = rvec
                fixed_tvec = tvec
                # Save fixed position to JSON
                fixed_position = {
                    "fixed_rvec": fixed_rvec.tolist(),
                    "fixed_tvec": fixed_tvec.tolist()
                }
                with open("fixed_position.json", "w") as json_file:
                    json.dump(fixed_position, json_file, indent=4)
                print("Fixed position saved!")
                return True
            else:
                print("Failed to estimate pose.")
    print("Charuco board not detected.")
    return False


def calculate_relative_position(frame):
    global fixed_rvec, fixed_tvec

    if fixed_rvec is None or fixed_tvec is None:
        print("Fixed position not saved. Please save it first.")
        return

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    marker_corners, marker_ids, _ = detector.detectMarkers(gray)

    if len(marker_ids) > 0:
        _, charuco_corners, charuco_ids = cv2.aruco.interpolateCornersCharuco(marker_corners, marker_ids, gray, board)
        if charuco_ids is not None and len(charuco_corners) > 3:
            # Initialize rvec and tvec
            rvec = np.zeros((3, 1), dtype=np.float64)
            tvec = np.zeros((3, 1), dtype=np.float64)

            retval, rvec, tvec = cv2.aruco.estimatePoseCharucoBoard(
                charuco_corners, charuco_ids, board, mtx, dist, rvec, tvec
            )

            if retval:
                # Calculate relative position
                relative_tvec = tvec - fixed_tvec
                relative_rmat, _ = cv2.Rodrigues(rvec - fixed_rvec)
                relative_rvec, _ = cv2.Rodrigues(relative_rmat)

                print(f"Relative Translation (X, Y, Z): {relative_tvec.flatten()}")
                print(f"Relative Rotation Vector: {relative_rvec.flatten()}")
                return relative_tvec, relative_rvec
            else:
                print("Failed to estimate pose.")
    print("Charuco board not detected.")
    return None

# Main loop for capturing and processing frames
cap = cv2.VideoCapture(0)  # Change 0 to your camera ID if needed
while True:
    ret, frame = cap.read()
    if not ret:
        break

    cv2.imshow("Camera", frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord('s'):  # Press 's' to save the fixed position
        if save_fixed_position(frame):
            print("Fixed position saved successfully.")
        else:
            print("Failed to detect Charuco board. Try again.")

    elif key == ord('r'):  # Press 'r' to calculate the relative position
        calculate_relative_position(frame)

    elif key == ord('q'):  # Press 'q' to quit
        break

cap.release()
cv2.destroyAllWindows()


Fixed position loaded from file.
Relative Translation (X, Y, Z): [-5.44987581e-04 -3.41331998e-05  5.01621987e-03]
Relative Rotation Vector: [-0.00262894 -0.00109593  0.00072063]
